In [3]:
import requests
import re
import pandas
import random
from bs4 import BeautifulSoup


USER_AGENTS = [
    "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; AcooBrowser; .NET CLR 1.1.4322; .NET CLR 2.0.50727)",
    "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.0; Acoo Browser; SLCC1; .NET CLR 2.0.50727; Media Center PC 5.0; .NET CLR 3.0.04506)",
    "Mozilla/4.0 (compatible; MSIE 7.0; AOL 9.5; AOLBuild 4337.35; Windows NT 5.1; .NET CLR 1.1.4322; .NET CLR 2.0.50727)",
    "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Win64; x64; Trident/5.0; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 2.0.50727; Media Center PC 6.0)",
    "Mozilla/5.0 (compatible; MSIE 8.0; Windows NT 6.0; Trident/4.0; WOW64; Trident/4.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 1.0.3705; .NET CLR 1.1.4322)",
    "Mozilla/4.0 (compatible; MSIE 7.0b; Windows NT 5.2; .NET CLR 1.1.4322; .NET CLR 2.0.50727; InfoPath.2; .NET CLR 3.0.04506.30)",
    "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN) AppleWebKit/523.15 (KHTML, like Gecko, Safari/419.3) Arora/0.3 (Change: 287 c9dfb30)",
    "Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.8.1.2pre) Gecko/20070215 K-Ninja/2.1.1",
    "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN; rv:1.9) Gecko/20080705 Firefox/3.0 Kapiko/3.0",
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.56 Safari/535.11",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_3) AppleWebKit/535.20 (KHTML, like Gecko) Chrome/19.0.1036.7 Safari/535.20",
    "Opera/9.80 (Macintosh; Intel Mac OS X 10.6.8; U; fr) Presto/2.9.168 Version/11.52",
]

def create_headers():
    headers = dict()
    headers["User-Agent"] = random.choice(USER_AGENTS)
    headers["Referer"] = "http://www.ke.com"
    return headers

# 请求网页数据函数
def get_html(url):        
    headers = create_headers()
    response = requests.get(url, timeout=10, headers=headers)
    html = response.content
    return html


In [ ]:
import math
# 开始获得需要的板块数据
total_page = 1
loupan_list = list()
url = 'https://cq.fang.ke.com/loupan/'

soup = BeautifulSoup(get_html(url), "lxml")
# 获得总的页数
page_box = soup.find_all('div', class_='page-box')[0]
matches = re.search('.*data-total-count="(\d+)".*', str(page_box))
total_page = int(math.ceil(int(matches.group(1)) / 10))
print(total_page, url)


# 从第一页开始,一直遍历到最后一页
headers = create_headers()
for i in range(1, total_page + 1):
    page = url + '/pg{0}'.format(i)
    print(page)
    soup = BeautifulSoup(get_html(page), "lxml")

    # 获得有小区信息的panel
    house_elements = soup.find_all('li', class_="resblock-list")
    for house_elem in house_elements:
        price = house_elem.find('span', class_="number")
        desc = house_elem.find('span', class_="desc")
        total = house_elem.find('div', class_="second")
        loupan = house_elem.find('a', class_='name')


        # 继续清理数据
        try:
            price = price.text.strip() + desc.text.strip()
        except Exception as e:
            price = '0'
        loupan = loupan.text.replace("\n", "")

        try:
            total = total.text.strip().replace(u'总价', '')
            total = total.replace(u'/套起', '')
        except Exception as e:
            total = '0'
        print("price:{0}, desc:{1}, total:{2}, loupan:{3}".format(price, desc, total, loupan))
    break

In [ ]:

import random
import requests
from bs4 import BeautifulSoup
import re
import math
from lxml import etree


chinese_city_district_dict = dict()
chinese_area_dict = dict()

base_site = 'https://cq.ke.com/'


def get_districts():
    url = base_site + 'xiaoqu/'
    
    root = etree.HTML(get_html(url))
    elements = root.xpath('///div[3]/div[1]/dl[2]/dd/div/div/a')
    en_names = list()
    ch_names = list()
    for element in elements:
        link = element.attrib['href']
        en_names.append(link.split('/')[-2])
        ch_names.append(element.text)


    # 打印区县英文和中文名列表
    for index, name in enumerate(en_names):
        chinese_city_district_dict[name] = ch_names[index]
    return en_names


def get_areas(district):
    page = base_site + "xiaoqu/{0}".format(district)
    areas = list()
    try:
        root = etree.HTML(get_html(page))
        links = root.xpath('//div[3]/div[1]/dl[2]/dd/div/div[2]/a')

        # 针对a标签的list进行处理
        for link in links:
            relative_link = link.attrib['href']
            # 去掉最后的"/"
            relative_link = relative_link[:-1]
            # 获取最后一节
            area = relative_link.split("/")[-1]
            # 去掉区县名,防止重复
            if area != district:
                chinese_area = link.text
                chinese_area_dict[area] = chinese_area
                # print(chinese_area)
                areas.append(area)
        return areas
    except Exception as e:
        print(e)


with open("sechouse.csv", "a+", encoding='utf-8') as f:
    # 开始获得需要的板块数据
    total_page = 1
    sec_house_list = list()
    districts = get_districts()
    for district in districts:
        arealist = get_areas(district)
        for area in arealist:
            # 中文区县
            chinese_district = chinese_city_district_dict.get(district, "")
            # 中文版块
            chinese_area = chinese_area_dict.get(area, "")
            page = base_site + 'ershoufang/{0}/'.format(area)
            if area in ['beibinlu', 'dashiba' , 'dazhulin', 'guanyinqiao','haierlu', 'hongensi','huahuiyuan','huangnibang','jiangbeizui','jiazhou','longtousi']:
                print("skip: ", chinese_area)
                continue

            soup = BeautifulSoup(get_html(page), "lxml")
            # 获得总的页数
            page_box = soup.find_all('div', class_='house-lst-page-box')[0]
            matches = re.search('.*"totalPage":(\d+).*', str(page_box))
            total_page = int(matches.group(1))
            print(chinese_district, chinese_area, total_page, page)
            
            # 从第一页开始,一直遍历到最后一页
            for i in range(1, total_page + 1):
                page = base_site + 'ershoufang/{0}/pg{1}'.format(area, i)

                if area == 'longxing' and i < 51:
                    print('skip:', page)

                soup = BeautifulSoup(get_html(page), "lxml")
                # 获得有小区信息的panel
                house_elements = soup.find_all('li', class_="clear")
                for house_elem in house_elements:
                    pic = house_elem.find('a', class_="img").find('img', class_="lj-lazy").get('data-original').strip()
                    name = house_elem.find('div', class_='title').text.replace("\n", "").replace(',',';').strip()
                    url =  house_elem.find('div', class_='title').find('a').get('href').replace("\n", "").strip()
                    position = house_elem.find('div', class_='positionInfo').text.replace("\n", "").strip()
                    desc = house_elem.find('div', class_="houseInfo").text.replace("\n", "").strip()
                    starInfo = house_elem.find('div', class_="followInfo").text.replace("\n", "").strip()
                    tag = house_elem.find('div', class_="tag").text.replace("\n", "").strip()
                    price = house_elem.find('div', class_="totalPrice").text.replace("\n", "").strip()
                    unit_price = house_elem.find('div', class_="unitPrice").text.replace("\n", "").strip()

                    item_text = chinese_district + ', ' + chinese_area + ', ' + position + ', ' + \
                        name + ', ' + \
                        desc + ', ' + \
                        price + ', ' + \
                        unit_price + ', ' + \
                        starInfo + ', ' + \
                        tag + ', ' + \
                        url + ', ' + \
                        pic
                    f.write(item_text + "\n")

In [17]:
html_str = '''<li class="clear">
</div></li>
'''

house_elem = BeautifulSoup(html_str, "lxml")

pic = house_elem.find('a', class_="img").find('img', class_="lj-lazy").get('data-original').strip()
name = house_elem.find('div', class_='title').text.replace("\n", "").replace(',',';').strip()
url =  house_elem.find('div', class_='title').find('a').get('href').replace("\n", "").strip()
position = house_elem.find('div', class_='positionInfo').text.replace("\n", "").strip()
desc = house_elem.find('div', class_="houseInfo").text.replace("\n", "").strip()
starInfo = house_elem.find('div', class_="followInfo").text.replace("\n", "").strip()
tag = house_elem.find('div', class_="tag").text.replace("\n", "").strip()
price = house_elem.find('div', class_="totalPrice").text.replace("\n", "").strip()
unit_price = house_elem.find('div', class_="unitPrice").text.replace("\n", "").strip()

text = position + ', ' + \
      name + ', ' + \
      desc + ', ' + \
      price + ', ' + \
      unit_price + ', ' + \
      starInfo + ', ' + \
      tag + ', ' + \
      url + ', ' + \
      pic
print(text)


东原D7一期, 户型方正 中间楼层 朝中庭 出门轻轨和商城新上, 中楼层                        (共33层)                                                            | 2011年建 |                    3室2厅 | 93平米                        | 北, 150万, 16,130元/平, 0人关注    / 6天前发布, 近地铁 VR看装修新上, https://cq.ke.com/ershoufang/106114871021.html, https://ke-image.ljcdn.com/110000-inspection/b2e5de34-6456-4789-bf85-72675c7b66e8_1000.jpg!m_fill,w_280,h_210,f_jpg?from=ke.com


In [ ]:
import requests
import pandas as pd
import json
import numpy as np

def get_loc():
    api_url = 'https://restapi.amap.com/v3/geocode/geo'
    
    # 写进自己的key, 由于请求条数较多, 可能超配额了, 所以用了两个key
    # key_list = ['你的第一个key', '你的第二个key']
    
    # 读取爬取下来的房价数据
    taz_df = pd.read_csv(r'../df_ershoufang.csv')
    
    lat_list = []
    lng_list = []
    for _, row in taz_df.iterrows():
        address_name = '陕西省西安市' + row['district'] + row['name']
        _i = np.random.randint(0, 2)
        para_dict = {'address': address_name, 'key': key_list[_i], 'output': 'JSON'}

        try:
            r = requests.get(api_url, params=para_dict)
            json_data = json.loads(r.text)
            if json_data['status'] == '1':
                geo = json_data['geocodes'][0]['location']
                longitude = geo.split(',')[0]
                latitude = geo.split(',')[1]
            else:
                longitude = np.nan
                latitude = np.nan
        except:
            longitude = np.nan
            latitude = np.nan
            continue
        lat_list.append(latitude)
        lng_list.append(longitude)
        print(_i)
        print(longitude, latitude)

    loc_df = pd.DataFrame({'lng': lng_list, 'lat': lat_list})
    taz_df = pd.concat([taz_df, loc_df], axis=1)

    taz_df.to_csv(r'house_price.csv', encoding='utf_8_sig', index=False)